In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import pandas as pd
import tensorflow as tf

# Dependencies

import pandas as pd
import numpy as np
import seaborn as sns
import os
import random
random.seed(1) 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

%pylab inline
%config InlineBackend.figure_formats = ['retina']

import warnings
from scipy import stats
warnings.filterwarnings('ignore')
sns.set()

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [5]:
# Prepare to read the CSV file from Google Drive
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# Define the link for the CSV file
link = 'https://drive.google.com/file/d/17EmL2q_206QaaiVju6kmF4OwMPXhul8R/view?usp=share_link'
 
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('diabetes_2021_health_indicators_BRFSS2021.csv') 

# read the file and chunk it to allow for faster processing
df_data=[]
df = pd.read_csv('diabetes_2021_health_indicators_BRFSS2021.csv',encoding='utf-8',chunksize=10000,iterator=True)

for chunk in df:
    df_data.append(chunk)

# Concatenate the chunks
df_data_df=pd.concat(df_data)

In [9]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_health = df_data_df

# Review the DataFrame
display(df_health)

,Diabetes,Pre-Diabetic,Afford Costs,Physical Health,Mental Health,Exercise 30 Days,Heart Attack,Heart Disease,Stroke,Lung Disease,...,Gender,Age,Overweight/Obese,Education,Income,Smoker,Binge Drinker,Heavy Drinker,Fruit>1/Day,Veggies>1/Day
0,0,0,0,0,0,0,0,0,0,1,...,2,6,0,2,3,3,0,0,1,1
1,1,0,0,1,1,0,0,1,0,0,...,2,6,1,2,2,4,0,0,1,2
2,1,0,0,1,0,1,0,0,0,0,...,2,5,1,2,5,4,1,0,1,1
3,0,0,0,1,1,0,0,0,0,1,...,1,6,0,3,4,3,0,0,2,2
4,0,0,0,0,0,1,0,0,0,0,...,2,6,1,2,5,3,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242883,1,0,0,1,1,1,0,0,0,0,...,1,6,0,1,2,4,1,0,1,1
242884,0,0,1,1,0,1,0,0,0,1,...,2,2,0,2,3,3,0,0,1,1
242885,0,0,0,1,1,1,0,0,0,0,...,1,4,1,4,6,4,0,0,1,1
242886,0,0,0,1,1,1,0,0,0,0,...,1,6,0,2,4,4,0,0,1,1


In [8]:
df_health.columns

Index(['Diabetes', 'Pre-Diabetic', 'Afford Costs', 'Physical Health',
       'Mental Health', 'Exercise 30 Days', 'Heart Attack', 'Heart Disease',
       'Stroke', 'Lung Disease', 'Kidney Disease', 'Difficulty Walking',
       'Race/Ethnicity', 'Overall Health', 'Health Plan', 'Physical Activity',
       'High BP', 'Gender', 'Age', 'Overweight/Obese', 'Education', 'Income',
       'Smoker', 'Binge Drinker', 'Heavy Drinker', 'Fruit>1/Day',
       'Veggies>1/Day'],
      dtype='object')

In [10]:
health_df = df_health[['Diabetes', 'Pre-Diabetic', 'Heart Attack', 'Heart Disease',
       'Stroke', 'Lung Disease', 'Kidney Disease', 'Difficulty Walking',
       'High BP','Age', 'Overweight/Obese', 'Income','Smoker', 'Binge Drinker', 'Fruit>1/Day',
       'Veggies>1/Day']]
health_df

,Diabetes,Pre-Diabetic,Heart Attack,Heart Disease,Stroke,Lung Disease,Kidney Disease,Difficulty Walking,High BP,Age,Overweight/Obese,Income,Smoker,Binge Drinker,Fruit>1/Day,Veggies>1/Day
0,0,0,0,0,0,1,0,0,0,6,0,3,3,0,1,1
1,1,0,0,1,0,0,0,0,1,6,1,2,4,0,1,2
2,1,0,0,0,0,0,0,0,1,5,1,5,4,1,1,1
3,0,0,0,0,0,1,0,1,0,6,0,4,3,0,2,2
4,0,0,0,0,0,0,0,1,1,6,1,5,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242883,1,0,0,0,0,0,0,0,1,6,0,2,4,1,1,1
242884,0,0,0,0,0,1,0,0,1,2,0,3,3,0,1,1
242885,0,0,0,0,0,0,0,0,0,4,1,6,4,0,1,1
242886,0,0,0,0,0,0,0,0,1,6,0,4,4,0,1,1


In [11]:
# Split our data into our features and target arrays
y = health_df["Diabetes"].values
X = health_df.drop("Diabetes",axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)

Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_feat = len(X_train[0])
hidden_nodes_1= 80
hidden_nodes_2= 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=num_input_feat, activation='relu'))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,741
Trainable params: 3,741
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
5693/5693 [==============================] - 10s 1ms/step - loss: 0.3198 - accuracy: 0.8701
Epoch 2/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3160 - accuracy: 0.8716
Epoch 3/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3155 - accuracy: 0.8720
Epoch 4/100
5693/5693 [==============================] - 7s 1ms/step - loss: 0.3152 - accuracy: 0.8720
Epoch 5/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3150 - accuracy: 0.8718
Epoch 6/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3148 - accuracy: 0.8718
Epoch 7/100
5693/5693 [==============================] - 7s 1ms/step - loss: 0.3146 - accuracy: 0.8720
Epoch 8/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3145 - accuracy: 0.8723
Epoch 9/100
5693/5693 [==============================] - 8s 1ms/step - loss: 0.3144 - accuracy: 0.8721
Epoch 10/100
5693/5693 [==============================] - 7s 1ms/step - 

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1898/1898 - 3s - loss: 0.5850 - accuracy: 0.8685 - 3s/epoch - 2ms/step
Loss: 0.585015594959259, Accuracy: 0.8685484528541565
